In [33]:
import os
import json
import pandas as pd 
import traceback

In [34]:
from langchain_openai import ChatOpenAI

In [35]:
from langchain_community.chat_models import ChatOpenAI

In [36]:
from dotenv import load_dotenv

load_dotenv() 

True

In [37]:
KEY=os.getenv("OPENAI_API_KEY")

In [38]:
llm= ChatOpenAI(openai_api_key=KEY,model_name= "gpt-3.5-turbo",temperature=0.3)

In [39]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [44]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [45]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [46]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [47]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [48]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [49]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [50]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [51]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [52]:
file_path= r"C:\Users\Jitheshreddy\mcqgen\data.txt"

In [53]:
file_path

'C:\\Users\\Jitheshreddy\\mcqgen\\data.txt'

In [54]:
with open(file_path, 'r', encoding='utf-8') as file:
    TEXT = file.read()


In [55]:
print(TEXT)

MKK
Unit 3
COMPUTER-BASED 
ASYMMETRIC-KEY CRYPTOGRAPHY 
ALGORITHMS
MKK
INTRODUCTION
• Symmetric-key cryptography is fast and efficient. 
• However, it suffers from a big disadvantage of the problem of key exchange. 
• The sender and the receiver of an encrypted message use the same key in 
symmetric-key cryptography, and it is very tough to agree upon a common key 
without letting anyone else know about it. 
• Asymmetric-key cryptography solves this problem. Here, each communicating party 
uses two keys to form a key pair—one key (the private key) remains with the party, 
and the other key (the public key) is shared with everybody.
MKK
BRIEF HISTORY OF ASYMMETRIC-KEY CRYPTOGRAPHY
In the mid-1970s, Whitfield Diffie, a student at the Stanford University met with Martin Hellman,
his professor, and the two began to think about the problem of key exchange. After some research
and complicated mathematical analysis, they came up with the idea of asymmetric-key cryptography.
Many experts belie

In [56]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"

In [57]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [58]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:MKK
Unit 3
COMPUTER-BASED 
ASYMMETRIC-KEY CRYPTOGRAPHY 
ALGORITHMS
MKK
INTRODUCTION
• Symmetric-key cryptography is fast and efficient. 
• However, it suffers from a big disadvantage of the problem of key exchange. 
• The sender and the receiver of an encrypted message use the same key in 
symmetric-key cryptography, and it is very tough to agree upon a common key 
without letting anyone else know about it. 
• Asymmetric-key cryptography solves this problem. Here, each communicating party 
uses two keys to form a key pair—one key (the private key) remains with the party, 
and the other key (the public key) is shared with everybody.
MKK
BRIEF HISTORY OF ASYMMETRIC-KEY CRYPTOGRAPHY
In the mid-1970s, Whitfield Diffie, a student at the Stanford University met with Martin Hellman,
his professor, and the two began to think about the problem of key exchange. After some research
and comp

In [59]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost(in doller):{cb.total_cost}")

Total Tokens:3829
Prompt Tokens:3419
Completion Tokens:410
Total Cost(in doller):0.0059485


In [60]:
response

{'text': 'MKK\nUnit 3\nCOMPUTER-BASED \nASYMMETRIC-KEY CRYPTOGRAPHY \nALGORITHMS\nMKK\nINTRODUCTION\n• Symmetric-key cryptography is fast and efficient. \n• However, it suffers from a big disadvantage of the problem of key exchange. \n• The sender and the receiver of an encrypted message use the same key in \nsymmetric-key cryptography, and it is very tough to agree upon a common key \nwithout letting anyone else know about it. \n• Asymmetric-key cryptography solves this problem. Here, each communicating party \nuses two keys to form a key pair—one key (the private key) remains with the party, \nand the other key (the public key) is shared with everybody.\nMKK\nBRIEF HISTORY OF ASYMMETRIC-KEY CRYPTOGRAPHY\nIn the mid-1970s, Whitfield Diffie, a student at the Stanford University met with Martin Hellman,\nhis professor, and the two began to think about the problem of key exchange. After some research\nand complicated mathematical analysis, they came up with the idea of asymmetric-key cry

In [61]:
quiz = response.get("quiz")

In [62]:
quiz=json.loads(quiz)

In [63]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [64]:
quiz=pd.DataFrame(quiz_table_data)

In [65]:
quiz

,MCQ,Choices,Correct
0,What is the main disadvantage of symmetric-key...,a: Key exchange problem | b: Slow encryption p...,a
1,Who are considered the fathers of asymmetric-k...,a: Whitfield Diffie and Martin Hellman | b: Ro...,a
2,What is the main advantage of asymmetric-key c...,a: Fast and efficient encryption process | b: ...,d
3,Which algorithm is based on the mathematical f...,a: Knapsack algorithm | b: RSA algorithm | c: ...,b
4,What type of attack involves an attacker trick...,a: Chosen-message Attack | b: Known-message At...,a


In [66]:
quiz.to_csv("machinelearning.csv",index=False)

In [67]:
from datetime import datetime

In [68]:
datetime.now().strftime("%m-%d-%Y %H:%M:%S")

'04-07-2024 18:28:51'